# Using deep models in retrieval

In [the featurization tutorial](https://github.com/tensorflow/recommenders/blob/main/docs/examples/featurization.ipynb) we incorporated multiple features into our models, but the models consist of only an embedding layer. We can add more dense layers to our models to increase their capacity.

Deep models with multiple layers can approximate more complex patterns and functions than models with only an embedding layer. Furthermore, with more layers, the learnability of the model might also improve. While model with one hidden layer can approximate any function in theory, in practice models with more hidden layers can learn to approximate complex functions more easily.

Nonetheless, complex models also have their disadvantages. More layers require more training epochs, and each training step would require more computation. It would also be harder for the gradients to propagate through models with more layers when updating model parameters. Furthermore, with more parameters, deep models might overfit or even simply memorize the training examples instead of learning a function that can generalize.

In this notebook we will build deep models with multiple layers and compare the results.

## Preliminaries

We first import the necessary packages.

In [3]:
import os
import tempfile

import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds

import tensorflow_recommenders as tfrs

In this tutorial we will use the models from [the featurization tutorial](https://github.com/tensorflow/recommenders/blob/main/docs/examples/featurization.ipynb) to generate embeddings. Hence we will only be using the user id, timestamp, and movie title features.

In [4]:
ratings = tfds.load("movielens/100k-ratings", split="train")
movies = tfds.load("movielens/100k-movies", split="train")

ratings = ratings.map(lambda x: {
    "movie_title": x["movie_title"],
    "user_id": x["user_id"],
    "timestamp": x["timestamp"],
})
movies = movies.map(lambda x: {
    "movie_title": x["movie_title"],
})

We bucketize the timestamp feature.

In [5]:
max_timestamp = ratings.map(lambda x: x["timestamp"]).reduce(
    tf.cast(0, tf.int64), tf.maximum).numpy().max()
min_timestamp = ratings.map(lambda x: x["timestamp"]).reduce(
    np.int64(1e9), tf.minimum).numpy().min()
timestamp_buckets = np.linspace(
    min_timestamp, max_timestamp, num=1000,
)

## Model definition

### Query model

We will define a generic query model that can have different architectures depending on the constructor arguments.

We will use the user model defined in [the featurization tutorial](https://github.com/tensorflow/recommenders/blob/main/docs/examples/featurization.ipynb) as a component of our query model. It will convert input examples into feature embeddings.

In [6]:
class UserModel(tf.keras.Model):
  
  def __init__(self):
    super().__init__()

    num_hashing_bins = 20_000

    self.user_embedding = tf.keras.Sequential([
        tf.keras.layers.experimental.preprocessing.Hashing(num_bins=num_hashing_bins),
        tf.keras.layers.Embedding(num_hashing_bins, 32),
    ])
    self.timestamp_embedding = tf.keras.Sequential([
        tf.keras.layers.experimental.preprocessing.Discretization(timestamp_buckets.tolist()),
        tf.keras.layers.Embedding(len(timestamp_buckets) + 2, 32),
    ])
    self.normalized_timestamp = tf.keras.layers.experimental.preprocessing.Normalization()

  def call(self, inputs):

    # Take the input dictionary, pass it through each input layer,
    # and concatenate the result.
    return tf.concat([
        self.user_embedding(inputs["user_id"]),
        self.timestamp_embedding(inputs["timestamp"]),
        self.normalized_timestamp(inputs["timestamp"]),
    ], axis=1)

In addition to the embedding model, we also add hidden layers according to the argument `hidden_layer_sizes` to make the query model deep and use activation functions specified by the argument `activation`. After adding the hidden layers, we add a projection layer to generate embeddings of dimensionality specified by `final_embedding_dimension`.

Note that we do not use any activation function on the projection layer. Using an activation function would limit the output space of the final embeddings and might negatively impact the performance of the model. For instance, if ReLUs are used in the projection layer, all components in the output embedding would be non-negative.

In [7]:
class QueryModel(tf.keras.Model):
  """Model for encoding user queries."""

  def __init__(
      self,
      final_embedding_dimension,
      hidden_layer_sizes=None,
      activation=None,
  ):
    """Initializes a query model for encoding user queries.

    Args:
      final_embedding_dimension:
        An integer representing the dimensionality of the final embedding. The
        model would add a final projection layer with
        `final_embedding_dimension` units to ensure that the output embedding
        has the specified number of dimensions.
      hidden_layer_sizes:
        A list of integers where the ith entry represents the number of units
        the ith layer contains.
      activation:
        A string representing the activation function to be used in all layers
        except the last layer. It will be passed to `tf.keras.layers.Dense` as
        an argument. If activation is None, then no activation function is used.
    
    Returns:
      A query model for encoding queries.
    """
    super(QueryModel, self).__init__()

    if hidden_layer_sizes is None:
      hidden_layer_sizes = []

    # We first use the user model for generating embeddings.
    self.embedding_model = UserModel()

    dense_layer_list = []
    # We now add the hidden layers.
    for layer_size in hidden_layer_sizes:
      dense_layer_list.append(
          tf.keras.layers.Dense(layer_size, activation=activation),
      )
    # We finally add a projection layer without any activation function.
    dense_layer_list.append(
        tf.keras.layers.Dense(final_embedding_dimension, activation=None),
    )
    self.dense_layers = tf.keras.Sequential(dense_layer_list, name='dense_layers')
    
  def call(self, inputs):
    feature_embedding = self.embedding_model(inputs)
    return self.dense_layers(feature_embedding)

### Candidate model

Since we are focusing on exploring different query models, we will keep the candidate model simple and use only the embedding model from [the featurization tutorial](https://github.com/tensorflow/recommenders/blob/main/docs/examples/featurization.ipynb). Note that this model generates embeddings with 64 dimensions.

In [8]:
class MovieModel(tf.keras.Model):
  
  def __init__(self):
    super().__init__()

    num_hashing_bins = 20_000
    max_tokens = 10_000

    self.title_embedding = tf.keras.Sequential([
      tf.keras.layers.experimental.preprocessing.Hashing(num_bins=num_hashing_bins),
      tf.keras.layers.Embedding(num_hashing_bins, 32)
    ])
    self.title_text_embedding = tf.keras.Sequential([
      tf.keras.layers.experimental.preprocessing.TextVectorization(max_tokens=max_tokens),
      tf.keras.layers.Embedding(max_tokens, 32)
    ])

  def call(self, inputs):
    return tf.concat([
        self.title_embedding(inputs["movie_title"]),
        # We average the embedding of individual words to get one embedding vector
        # per title.
        tf.reduce_mean(self.title_text_embedding(inputs["movie_title"]), axis=-2),
    ], axis=1)

### Combined model

We now define a combined model that takes a query model and a candidate model as arguments.

In [9]:
class MovielensModel(tfrs.models.Model):

  def __init__(self, query_model, candidate_model):
    super().__init__()
    self.query_model: tf.keras.Model = query_model
    self.candidate_model: tf.keras.Model = candidate_model
    self.task = tfrs.tasks.Retrieval(
        metrics=tfrs.metrics.FactorizedTopK(
            candidates=movies.batch(128).map(self.candidate_model),
        ),
    )

  def compute_loss(self, features, training=False):
    # We only pass the user id and timestamp features into the query model. This
    # is to ensure that the training inputs would have the same keys as the
    # query inputs. Otherwise the discrepancy in input structure would cause an
    # error when loading the query model after saving it.
    query_embeddings = self.query_model({
        "user_id": features["user_id"],
        "timestamp": features["timestamp"],
    })
    positive_movie_embeddings = self.candidate_model({
        "movie_title": features["movie_title"],
    })
    return self.task(query_embeddings, positive_movie_embeddings, training=False)

## Training the model

### Prepare the data

We first split the data into a training set and a testing set.

In [10]:
tf.random.set_seed(42)
shuffled = ratings.shuffle(100_000, seed=42, reshuffle_each_iteration=False)

train = shuffled.take(80_000)
test = shuffled.skip(80_000).take(20_000)

cached_train = train.shuffle(100_000).batch(8192).cache()
cached_test = test.batch(4096).cache()

### Model with one dense layer

We now define the dimensionalities of the final embeddings below. Since `MovieModel` generates embeddings with 64 dimensions, we want our query models to generate 64 dimensional embeddings as well.

In [11]:
final_embedding_dimension = 64

We now define a simple model with only a projection layer and no hidden layer. The activation function does not matter since the projection layer does not have any activation function. 

In [12]:
one_layer_query_model = QueryModel(
    final_embedding_dimension,
)
one_layer_candidate_model = MovieModel()
one_layer_candidate_model.title_text_embedding.layers[0].adapt(
    movies.map(lambda x: x["movie_title"]),
)
one_layer_model = MovielensModel(
    one_layer_query_model,
    one_layer_candidate_model,
)
one_layer_model.compile(optimizer=tf.keras.optimizers.Adagrad(0.1))

In [13]:
one_layer_model.fit(cached_train, epochs=3)

Epoch 1/3
10/10 [==============================] - 21s 2s/step - factorized_top_k: 2.1250e-04 - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_10_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_50_categorical_accuracy: 1.2500e-04 - factorized_top_k/top_100_categorical_accuracy: 9.3750e-04 - loss: 639008857367.2727 - regularization_loss: 0.0000e+00 - total_loss: 639008857367.2727
Epoch 2/3
10/10 [==============================] - 18s 2s/step - factorized_top_k: 0.0015 - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 6.2500e-05 - factorized_top_k/top_10_categorical_accuracy: 8.7500e-05 - factorized_top_k/top_50_categorical_accuracy: 9.0000e-04 - factorized_top_k/top_100_categorical_accuracy: 0.0066 - loss: 494642662306.9091 - regularization_loss: 0.0000e+00 - total_loss: 494642662306.9091
Epoch 3/3
10/10 [==============================]

### Model with three dense layers and no activation function

We now create and train a model with two hidden layers of size 64 and one projection layer and no activation function.

In [14]:
three_layer_query_model = QueryModel(
    final_embedding_dimension,
    hidden_layer_sizes=[64, 64],
    activation=None,
)
three_layer_candidate_model = MovieModel()
three_layer_candidate_model.title_text_embedding.layers[0].adapt(
    movies.map(lambda x: x["movie_title"]),
)
three_layer_model = MovielensModel(
    three_layer_query_model,
    three_layer_candidate_model,
)
three_layer_model.compile(optimizer=tf.keras.optimizers.Adagrad(0.1))

In [15]:
three_layer_model.fit(cached_train, epochs=3)

Epoch 1/3
10/10 [==============================] - 20s 2s/step - factorized_top_k_1: 0.0018 - factorized_top_k_1/top_1_categorical_accuracy: 1.2500e-05 - factorized_top_k_1/top_5_categorical_accuracy: 8.7500e-05 - factorized_top_k_1/top_10_categorical_accuracy: 2.5000e-04 - factorized_top_k_1/top_50_categorical_accuracy: 0.0023 - factorized_top_k_1/top_100_categorical_accuracy: 0.0064 - loss: 43854249161821.0938 - regularization_loss: 0.0000e+00 - total_loss: 43854249161821.0938
Epoch 2/3
10/10 [==============================] - 21s 2s/step - factorized_top_k_1: 0.0052 - factorized_top_k_1/top_1_categorical_accuracy: 6.2500e-05 - factorized_top_k_1/top_5_categorical_accuracy: 3.1250e-04 - factorized_top_k_1/top_10_categorical_accuracy: 0.0011 - factorized_top_k_1/top_50_categorical_accuracy: 0.0067 - factorized_top_k_1/top_100_categorical_accuracy: 0.0177 - loss: 3639035518231.2729 - regularization_loss: 0.0000e+00 - total_loss: 3639035518231.2729
Epoch 3/3
10/10 [=====================

### Model with five dense layers and no activation function

We now create a model with four hidden layers of sizes 128, 128, 64, 64, and a projection layer and no activation function.

In [16]:
five_layer_query_model = QueryModel(
    final_embedding_dimension,
    hidden_layer_sizes=[128, 128, 64, 64],
    activation=None,
)
five_layer_candidate_model = MovieModel()
five_layer_candidate_model.title_text_embedding.layers[0].adapt(
    movies.map(lambda x: x["movie_title"]),
)
five_layer_model = MovielensModel(
    five_layer_query_model,
    five_layer_candidate_model,
)
five_layer_model.compile(optimizer=tf.keras.optimizers.Adagrad(0.1))

In [17]:
five_layer_model.fit(cached_train, epochs=3)

Epoch 1/3
10/10 [==============================] - 20s 2s/step - factorized_top_k_2: 0.0017 - factorized_top_k_2/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k_2/top_5_categorical_accuracy: 0.0000e+00 - factorized_top_k_2/top_10_categorical_accuracy: 2.1250e-04 - factorized_top_k_2/top_50_categorical_accuracy: 0.0026 - factorized_top_k_2/top_100_categorical_accuracy: 0.0057 - loss: 5755433769405347.0000 - regularization_loss: 0.0000e+00 - total_loss: 5755433769405347.0000
Epoch 2/3
10/10 [==============================] - 19s 2s/step - factorized_top_k_2: 0.0112 - factorized_top_k_2/top_1_categorical_accuracy: 5.0000e-05 - factorized_top_k_2/top_5_categorical_accuracy: 4.6250e-04 - factorized_top_k_2/top_10_categorical_accuracy: 0.0022 - factorized_top_k_2/top_50_categorical_accuracy: 0.0168 - factorized_top_k_2/top_100_categorical_accuracy: 0.0367 - loss: 671937805061026.8750 - regularization_loss: 0.0000e+00 - total_loss: 671937805061026.8750
Epoch 3/3
10/10 [=============

### Model with three dense layers and ReLU activation function

We now create a train a model with two hidden layers of size 64 and a projection layer. We use ReLUs in the hidden layers.

In [18]:
three_layer_relu_query_model = QueryModel(
    final_embedding_dimension,
    hidden_layer_sizes=[64, 64],
    activation="relu",
)
three_layer_relu_candidate_model = MovieModel()
three_layer_relu_candidate_model.title_text_embedding.layers[0].adapt(
    movies.map(lambda x: x["movie_title"]),
)
three_layer_relu_model = MovielensModel(
    three_layer_relu_query_model,
    three_layer_relu_candidate_model,
)
three_layer_relu_model.compile(optimizer=tf.keras.optimizers.Adagrad(0.1))

In [19]:
three_layer_relu_model.fit(cached_train, epochs=3)

Epoch 1/3
10/10 [==============================] - 20s 2s/step - factorized_top_k_3: 0.0148 - factorized_top_k_3/top_1_categorical_accuracy: 2.3750e-04 - factorized_top_k_3/top_5_categorical_accuracy: 0.0015 - factorized_top_k_3/top_10_categorical_accuracy: 0.0020 - factorized_top_k_3/top_50_categorical_accuracy: 0.0236 - factorized_top_k_3/top_100_categorical_accuracy: 0.0465 - loss: 705679185474.3508 - regularization_loss: 0.0000e+00 - total_loss: 705679185474.3508
Epoch 2/3
10/10 [==============================] - 18s 2s/step - factorized_top_k_3: 0.0135 - factorized_top_k_3/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k_3/top_5_categorical_accuracy: 0.0000e+00 - factorized_top_k_3/top_10_categorical_accuracy: 0.0000e+00 - factorized_top_k_3/top_50_categorical_accuracy: 0.0234 - factorized_top_k_3/top_100_categorical_accuracy: 0.0443 - loss: 70371.9474 - regularization_loss: 0.0000e+00 - total_loss: 70371.9474
Epoch 3/3
10/10 [==============================] - 18s 2s/step

## Comparing the models

We now evaluate the four models and compare their results.

In [20]:
one_layer_model.evaluate(cached_test, return_dict=True)

5/5 [==============================] - 4s 706ms/step - factorized_top_k: 0.0291 - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 0.0010 - factorized_top_k/top_10_categorical_accuracy: 0.0023 - factorized_top_k/top_50_categorical_accuracy: 0.0413 - factorized_top_k/top_100_categorical_accuracy: 0.1006 - loss: 156813167274.6667 - regularization_loss: 0.0000e+00 - total_loss: 156813167274.6667


{'factorized_top_k': array([0.     , 0.00105, 0.0023 , 0.04135, 0.1006 ], dtype=float32),
 'factorized_top_k/top_100_categorical_accuracy': 0.1005999967455864,
 'factorized_top_k/top_10_categorical_accuracy': 0.002300000051036477,
 'factorized_top_k/top_1_categorical_accuracy': 0.0,
 'factorized_top_k/top_50_categorical_accuracy': 0.041349999606609344,
 'factorized_top_k/top_5_categorical_accuracy': 0.0010499999625608325,
 'loss': 153423282176.0,
 'regularization_loss': 0,
 'total_loss': 153423282176.0}

In [21]:
three_layer_model.evaluate(cached_test, return_dict=True)

5/5 [==============================] - 3s 667ms/step - factorized_top_k_1: 0.0327 - factorized_top_k_1/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k_1/top_5_categorical_accuracy: 7.0000e-04 - factorized_top_k_1/top_10_categorical_accuracy: 0.0041 - factorized_top_k_1/top_50_categorical_accuracy: 0.0483 - factorized_top_k_1/top_100_categorical_accuracy: 0.1102 - loss: 124413053610.6667 - regularization_loss: 0.0000e+00 - total_loss: 124413053610.6667


{'factorized_top_k_1': array([0.     , 0.0007 , 0.0041 , 0.04835, 0.11015], dtype=float32),
 'factorized_top_k_1/top_100_categorical_accuracy': 0.11015000194311142,
 'factorized_top_k_1/top_10_categorical_accuracy': 0.004100000020116568,
 'factorized_top_k_1/top_1_categorical_accuracy': 0.0,
 'factorized_top_k_1/top_50_categorical_accuracy': 0.04834999889135361,
 'factorized_top_k_1/top_5_categorical_accuracy': 0.000699999975040555,
 'loss': 89817063424.0,
 'regularization_loss': 0,
 'total_loss': 89817063424.0}

In [22]:
five_layer_model.evaluate(cached_test, return_dict=True)

5/5 [==============================] - 3s 641ms/step - factorized_top_k_2: 0.0222 - factorized_top_k_2/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k_2/top_5_categorical_accuracy: 4.5000e-04 - factorized_top_k_2/top_10_categorical_accuracy: 0.0038 - factorized_top_k_2/top_50_categorical_accuracy: 0.0349 - factorized_top_k_2/top_100_categorical_accuracy: 0.0719 - loss: 20117024582314.6680 - regularization_loss: 0.0000e+00 - total_loss: 20117024582314.6680


{'factorized_top_k_2': array([0.     , 0.00045, 0.00385, 0.03495, 0.0719 ], dtype=float32),
 'factorized_top_k_2/top_100_categorical_accuracy': 0.07190000265836716,
 'factorized_top_k_2/top_10_categorical_accuracy': 0.0038499999791383743,
 'factorized_top_k_2/top_1_categorical_accuracy': 0.0,
 'factorized_top_k_2/top_50_categorical_accuracy': 0.034949999302625656,
 'factorized_top_k_2/top_5_categorical_accuracy': 0.00044999999227002263,
 'loss': 18365051568128.0,
 'regularization_loss': 0,
 'total_loss': 18365051568128.0}

In [23]:
three_layer_relu_model.evaluate(cached_test, return_dict=True)

5/5 [==============================] - 3s 636ms/step - factorized_top_k_3: 0.0156 - factorized_top_k_3/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k_3/top_5_categorical_accuracy: 0.0000e+00 - factorized_top_k_3/top_10_categorical_accuracy: 0.0000e+00 - factorized_top_k_3/top_50_categorical_accuracy: 0.0308 - factorized_top_k_3/top_100_categorical_accuracy: 0.0472 - loss: 32590.7090 - regularization_loss: 0.0000e+00 - total_loss: 32590.7090


{'factorized_top_k_3': array([0.     , 0.     , 0.     , 0.0308 , 0.04725], dtype=float32),
 'factorized_top_k_3/top_100_categorical_accuracy': 0.04724999889731407,
 'factorized_top_k_3/top_10_categorical_accuracy': 0.0,
 'factorized_top_k_3/top_1_categorical_accuracy': 0.0,
 'factorized_top_k_3/top_50_categorical_accuracy': 0.030799999833106995,
 'factorized_top_k_3/top_5_categorical_accuracy': 0.0,
 'loss': 29628.37109375,
 'regularization_loss': 0,
 'total_loss': 29628.37109375}

For all values of k except five, the three layer model with no activation function has the highest top k categorical accuracy. This shows that having multiple layers in a model can improve its performance as the three layer model outperformed the one layer model. However, the performance of the five layer model shows that a more complex model does not always guarantee better performance. As mentioned above, complex models require more training epochs. In this case, three epochs might not be sufficient for a five layer model.

Note that the three layer model with ReLUs also performs worse than the three layer model without any activation function. This shows how activation can also affect model performance.

## Serving the model

We now serve our best performing model, the three layer model. In a simple recommendation model, when receiving a query, we simply compute the embedding of the query and find the movies with the closest embeddings to that query. For efficiency, we can precompute the embeddings of all candidates.

In [24]:
movie_embeddings = movies.batch(1_000).map(lambda x: three_layer_model.candidate_model(x))

To create a recommendation model that takes a query and returns the top candidates, we use the `ann` module in TFRS. Since the number of movies in the dataset is small, we can use the `BruteForce` layer to look for top candidates. We pass in the query model as an argument for encoding input queries.

In [25]:
serving_model = tfrs.layers.ann.BruteForce(three_layer_model.query_model)

We then let the recommendation model index the candidates by calling `index` and pass in the movie embeddings we precomputed and the movie titles as identifiers.

In [26]:
serving_model.index(
    candidates=movie_embeddings,
    identifiers=movies.batch(1_000).map(lambda x: x["movie_title"]),
)

We can query the model for recommendations by passing in a dictionary of features.

In [27]:
scores, titles = serving_model(
    {"user_id": np.array(["42"]), "timestamp": np.array([879024327])},
    num_candidates=3,
)
for i, title in enumerate(titles[0].numpy().tolist()):
  print("{:d}. {:s}".format(i + 1, str(title)))

1. b'Shall We Dance? (1937)'
2. b'Dear God (1996)'
3. b'Blue Angel, The (Blaue Engel, Der) (1930)'


We can save the model using `tf.saved_model.save`.

In [28]:
tmp = tempfile.TemporaryDirectory()
path = os.path.join(tmp.name, "s_model")
tf.saved_model.save(serving_model, path)

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.


Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.


Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.


Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.


INFO:tensorflow:Assets written to: /tmp/tmpnpn52hxo/s_model/assets


INFO:tensorflow:Assets written to: /tmp/tmpnpn52hxo/s_model/assets


We can then load the model using `tf.keras.models.load_model`.

In [29]:
loaded_model = tf.keras.models.load_model(path)

We can query the loaded model for recommendations by passing in a dictionary of features.

In [30]:
scores, titles = loaded_model(
    {"user_id": np.array(["42"]), "timestamp": np.array([879024327])}
)
for i, title in enumerate(titles[0].numpy().tolist()):
  print("{:d}. {:s}".format(i + 1, str(title)))

1. b'Shall We Dance? (1937)'
2. b'Dear God (1996)'
3. b'Blue Angel, The (Blaue Engel, Der) (1930)'
4. b'Bottle Rocket (1996)'
5. b'His Girl Friday (1940)'
6. b'Set It Off (1996)'
7. b'Wild America (1997)'
8. b'Cape Fear (1962)'
9. b'Sleepers (1996)'
10. b'Soul Food (1997)'


## Next Steps

In this tutorial we expanded our retrieval model with dense layers and activation functions. To see how to create a model that can perform not only retrieval tasks but also rating tasks, take a look at [the multitask tutorial](https://github.com/tensorflow/recommenders/blob/main/docs/examples/multitask.ipynb).